In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [ ]:
pkd=pd.read_csv('/content/parkinsons.csv')
pkd.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [ ]:
pkd=pkd.drop(columns=['name'])

In [ ]:
x=pkd.drop('status',axis=1)
y=pkd['status']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)

StandardScaler()

In [ ]:
x_sca_train=scaler.fit_transform(x_train)
x_sca_test=scaler.fit_transform(x_test)

In [ ]:
!pip install optuna
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
def objective(trial):
    """Objective function for Optuna optimization"""
    C = trial.suggest_loguniform('C', 1e-3, 1e3)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3  # Only used for poly kernel
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, random_state=42)

    score = cross_val_score(model, x_sca_train, y_train, cv=5, scoring='accuracy').mean()
    return score

# Optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# Print best hyperparameters
print("Best parameters:", study.best_params)
print("Best score:", study.best_value)


[I 2025-03-16 17:47:10,357] A new study created in memory with name: no-name-8bc9c93b-19fb-48f6-a91e-cf1b7a28e61d
<ipython-input-70-3a7ab17b02ed>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-3, 1e3)
[I 2025-03-16 17:47:10,383] Trial 0 finished with value: 0.7820564516129032 and parameters: {'C': 0.16881743931410767, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 0 with value: 0.7820564516129032.
<ipython-input-70-3a7ab17b02ed>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-3, 1e3)
[I 2025-03-16 17:47:10,404] Trial 1 finished with value: 0.7820564516129032 and parameters: {'C': 0.00164

Best parameters: {'C': 97.86721247052272, 'kernel': 'rbf', 'gamma': 'auto'}
Best score: 0.9171370967741936


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
model1=SVC(C=108.08518868710962, kernel='rbf', gamma='auto')
model1.fit(x_sca_train,y_train)
e1=model1.predict(x_sca_train)
f1=model1.predict(x_sca_test)
print(accuracy_score(y_train,e1))
print(accuracy_score(y_test,f1))

1.0
0.8461538461538461


In [ ]:
#evaluting by input

input_data=(117.226	,123.925,	106.656,	0.00417,	0.00004,	0.00186,	0.0027,	0.00558,	0.01909,	0.171,	0.00864,	0.01223,	0.01949,	0.02592,	0.00955,	23.079,	0.603515,	0.669565,	-5.61907,	0.191576,	2.027228,	0.215724)
#convert tuple into numpy array
input_data_as_numpy_array=np.asarray(input_data)
#reshape the array because eta na hole model main dataset er row column ke dhore boshe thakbe. tai 1 ta data point er jonno reshape kora lage

input_data_reshaped=input_data_as_numpy_array.reshape(1,-1)

#standradized the data

prediction=model1.predict(input_data_reshaped)
print(prediction)
if prediction[0]==0:
  print('Healthy')
else:
  print('Parkinsons affected')


[0]
Healthy


In [ ]:
#saving the model
import pickle

In [ ]:
filename='trained_model.sav'

pickle.dump(model1,open(filename,'wb'))

In [ ]:
#loading the saved model
loaded_model= pickle.load(open('trained_model.sav','rb'))

In [ ]:
#evaluting by input

input_data=(117.226	,123.925,	106.656,	0.00417,	0.00004,	0.00186,	0.0027,	0.00558,	0.01909,	0.171,	0.00864,	0.01223,	0.01949,	0.02592,	0.00955,	23.079,	0.603515,	0.669565,	-5.61907,	0.191576,	2.027228,	0.215724)
#convert tuple into numpy array
input_data_as_numpy_array=np.asarray(input_data)
#reshape the array because eta na hole model main dataset er row column ke dhore boshe thakbe. tai 1 ta data point er jonno reshape kora lage

input_data_reshaped=input_data_as_numpy_array.reshape(1,-1)

#difference is here at loaded_model keyword here we store all value of traning
prediction=loaded_model.predict(input_data_reshaped)
print(prediction)
if prediction[0]==0:
  print('Healthy')
else:
  print('Parkinsons affected')

[0]
Healthy
